# Spark init

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
sc = SparkContext(conf=conf)

# 1. feladat

In [71]:
def getYear(line):
    return str(int(line.split(",")[1])-int(line.split(",")[2]))
sc.textFile("oscar_age_male.csv")\
.filter(lambda line: line!="")\
.map(getYear)\
.map(lambda y: (y,1))\
.reduceByKey(lambda a,b: a+b)\
.sortBy(lambda x: x[1], False)\
.take(1)

[('1938', 5)]

# 2. feladat - sentiment

In [15]:
pos_words = ['great', 'masterpiece', 'awesome', 'enjoyable']
neg_words = ['garbage', 'worst', 'bad', 'boring']
pos_count = sc.accumulator(0)
neg_count = sc.accumulator(0)
neu_count = sc.accumulator(0)

def measLine(line):
    pos = 0
    neg = 0
    for word in line.split(" "):
        if word in pos_words:
            pos +=1
        elif word in neg_words:
            neg +=1
    if pos>neg:
        return 1
    elif neg>pos:
        return -1
    else:
        return 0

def countSentiment(i):
    if i==1: 
        pos_count.add(1)
    elif i==-1: 
        neg_count.add(1)
    elif i==0: 
        neu_count.add(1)

lines = sc.textFile("sentiment_input.txt")\
.map(measLine)\
.foreach(countSentiment)

print(('positive',pos_count.value),('negative',neg_count.value),('neutral',neu_count.value))

('positive', 2) ('negative', 4) ('neutral', 17)
